# Demo - Query, Key, and Value Vectors

In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
tokens = ["she", "gave", "the", "dog", "its", "food"]

In [ ]:
torch.manual_seed(2)

embedding_dim = 12
embeddings = {
    token: torch.randn(embedding_dim)
    for token in tokens
}

In [ ]:
W_Q = torch.randn(embedding_dim, embedding_dim)
W_K = torch.randn(embedding_dim, embedding_dim)
W_V = torch.randn(embedding_dim, embedding_dim)

Q = {t: embeddings[t] @ W_Q for t in tokens}
K = {t: embeddings[t] @ W_K for t in tokens}
V = {t: embeddings[t] @ W_V for t in tokens}

In [ ]:
query_token = "she"
query_vector = Q[query_token]

In [ ]:
scores = {}

for token in tokens:
    scores[token] = torch.dot(query_vector, K[token]).item()


In [ ]:
df = pd.DataFrame({
    "Token": tokens,
    "Query·Key Score": [scores[t] for t in tokens]
}).sort_values("Query·Key Score", ascending=False)

df

In [ ]:
raw_scores = torch.tensor([scores[t] for t in tokens])
attention_weights = F.softmax(raw_scores, dim=0)

attention_df = pd.DataFrame({
    "Token": tokens,
    "Attention Weight": attention_weights.detach().numpy()
}).sort_values("Attention Weight", ascending=False)

attention_df

In [ ]:
updated_she = torch.zeros(embedding_dim)

for i, token in enumerate(tokens):
    updated_she += attention_weights[i] * V[token]


In [ ]:
def cosine_similarity(a, b):
    return F.cosine_similarity(a.unsqueeze(0), b.unsqueeze(0)).item()

print("Similarity (original → updated 'she'):",
      cosine_similarity(embeddings["she"], updated_she))
